In [1]:
import pandas as pd
import numpy as np
import os
import simplejson
import json

In [3]:
def load_file(path):
    file_path = path
    df = pd.read_csv(file_path)
    #Automatic fixes
    df = df.replace(r'^\s*$', np.nan, regex=True) #replaces empty strings spacess with NaN
    return df


In [9]:


df = load_file('/Users/brandon/Desktop/combined_missing_data.csv')

df.drop(columns=['Unnamed:0', 'source_row'], inplace=True)



In [11]:

import random
import string
letters = string.ascii_lowercase


def random_boolean_list(length):
    return [random.choice([True, False]) for i in range(length)]

def random_string_list(length):
    values = ['a','b','c']
    return [random.choice(values) for i in range(length)]

def random_int_list(length):
    return [random.randint(0, 100) for i in range(length)]

def random_float_list(length):
    return [random.uniform(0, 100) for i in range(length)]

def random_column(rows):
    choice = [random_boolean_list, random_string_list, random_int_list, random_float_list]
    return random.choice(choice)(rows)

def random_dataframe(row_count, col_count):
    columns = [''.join(random.choice(letters) for i in range(7)) for i in range(col_count - 1)]
    df = pd.DataFrame()
    for column in columns:
        df[column] = random_column(row_count)
    df['target'] = random_boolean_list(row_count)
    return df

rfcuwwc


In [23]:
random_dataframe(300, 20).to_csv('/Users/brandon/Desktop/random200x20_combine.csv')

